In [13]:
import numpy as np
import gym 
import time 
import math 
import pygame
#from keras.preprocessing import Image
from PIL import Image
from IPython import display as ipythondisplay

In [14]:
pygame.init()
DISPLAYSURF=pygame.display.set_mode((500,500),0,32)
clock=pygame.time.Clock()
pygame.display.flip()

In [15]:
env = gym.make("CartPole-v1",render_mode="rgb_array")
#print(env.action_space.n)

In [35]:
Learning_Rate = 0.1
Discount = 0.95
nepisodes = 10000
total = 0
total_reward = 0
prev_reward = 0

Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

eps = 1

eps_decay_value = 0.99995

In [34]:
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape
#q_table[10]

(30, 30, 50, 50, 2)

In [31]:
def get_discrete_state(state):
    discrete_state = state[0]/np_array_win_size+ np.array([15,10,1,10])
    return tuple(discrete_state.astype(int))

In [32]:
env.reset()
#print(env.render())

(array([ 0.03127143, -0.00877699, -0.03635814,  0.03389557], dtype=float32),
 {})

In [36]:
#pygame.init()
  
# CREATING CANVAS
#canvas = pygame.display.set_mode((500, 500))
env.reset()
env.render()
for episode in range (nepisodes + 1): 
    t0 = time.time()
    discrete_state = get_discrete_state(env.reset())
    done = False
    episode_reward = 0 

    if episode % 1000 == 0: 
        print("Episode: " + str(episode))

    while not done: 

        if np.random.random() > eps:
           if(-1<discrete_state[0]<30 and -1<discrete_state[1]<30 and -1<discrete_state[2]<50 and -1<discrete_state[3]<50):
            action = np.argmax(q_table[discrete_state]) 
        else:

            action = np.random.randint(0, env.action_space.n) 
        new_state, reward, done, _, __ = env.step(action)

        episode_reward += reward 

        new_discrete_state = get_discrete_state(new_state)

        if episode % 1000 == 0: 
            
            image=env.render()
            image=Image.fromarray(image,'RGB')
            mode,size,data=image.mode,image.size,image.tobytes()
            image=pygame.image.fromstring(data,size,mode)
            DISPLAYSURF.blit(image,(0,0))
            pygame.display.update()
            clock.tick(100)

            if done:
             break



        if not done:
            #update q-table
          if(-1<new_discrete_state[0]<30 and -1<new_discrete_state[1]<30 and -1<new_discrete_state[2]<50 and -1<new_discrete_state[3]<50):
            max_future_q = np.max(q_table[new_discrete_state])
          if(-1<discrete_state[0]<30 and -1<discrete_state[1]<30 and -1<discrete_state[2]<50 and -1<discrete_state[3]<50):
            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - Learning_Rate) * current_q + Learning_Rate * (reward + Discount * max_future_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    if eps > 0.05: 
        if episode_reward > prev_reward and episode > 1000:
            eps= math.pow(eps_decay_value, episode - 1000)

            if episode % 500 == 0:
                print("Epsilon: " + str(eps))

    t1 = time.time() 
    episode_total = t1 - t0 
    total = total + episode_total

    total_reward += episode_reward 
    prior_reward = episode_reward

    if episode % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

ipythondisplay.clear_output(wait=True)


Episode: 0
Time Average: 0.00019267773628234863
Mean Reward: 0.019
Episode: 1000
Time Average: 0.0004557759761810303
Mean Reward: 22.392
Epsilon: 0.9753093024395111
Episode: 2000
Epsilon: 0.9512282354250458
Time Average: 0.0006599957942962646
Mean Reward: 22.8
Epsilon: 0.9277417467531685
Episode: 3000
Epsilon: 0.9048351558698463
Time Average: 0.0006038024425506592
Mean Reward: 22.518
Epsilon: 0.8824941446941661
Episode: 4000
Epsilon: 0.8607047486686201
Time Average: 0.0005574064254760742
Mean Reward: 21.313
Epsilon: 0.8394533480303666
Episode: 5000
Epsilon: 0.818726659298009
Time Average: 0.00038216757774353025
Mean Reward: 21.798
Epsilon: 0.7985117269685725
Episode: 6000
Epsilon: 0.7787959154194878
Time Average: 0.000531834602355957
Mean Reward: 21.476
Epsilon: 0.7595669010105212
Episode: 7000
Epsilon: 0.7408126643807126
Time Average: 0.0006428201198577881
Mean Reward: 20.519
Epsilon: 0.7225214829355084
Episode: 8000
Epsilon: 0.7046819235193919
Time Average: 0.000621946096420288
Mean 

In [ ]:
env.close()
pygame.quit()